In [1]:
import csv
import pandas as pd
import numpy as np
import requests
from IPython.core.display import HTML
from selenium import webdriver
from bs4 import BeautifulSoup
import timeit
import urllib2
import seaborn as sb
import matplotlib.pyplot as plt
import json
%matplotlib inline


In [2]:
###Scraping Data (Takes a couple minutes)
salary = []
company = []
location = []
jobs = []
jobdescription = []
for i in range(31):
    url = "http://www.careerbuilder.com/jobs-data-science?page_number=1&pay=20"
    index = 59
    char = i
    charplus = i + 1
    charplusstring = str(charplus)
    url2 = url[:index] + charplusstring + url[index + 1:]
    rm = requests.get(url2)
    soupm = BeautifulSoup(rm.content, "lxml")
    
    ###Appends job titles
    for d in soupm.findAll(class_='job-title'):
        jobs.append(d.text)
        
    ###Appends job descriptions
    for d in soupm.findAll(class_='job-description show-for-medium-up'):
        jobdescription.append(d.text)
    
    ###Appends salary, company, location.
    z = 0
    for d in soupm.findAll(class_='job-text'):
        if z % 3 == 0:
            salary.append(d.text)
        elif z % 3 == 1:
            company.append(d.text)
        else:
            location.append(d.text)
        
        z += 1
        

In [9]:
###Returning the lists to check
#salary
#company
#location
#jobs
#jobdescription

In [3]:
###Zipping them all together
info = zip(salary, company, location, jobs, jobdescription)

In [4]:
###Putting in DataFrame
df = pd.DataFrame(info)

In [5]:
###Naming Columns
df.columns = ('salary', 'company', 'location', 'title', 'description')

In [6]:
###Breaking location into State/City
df['state'] = [(x[-3:-1]) for x in df['location']]

In [7]:
###Breaking location into City
df['delete'] = df['location'].str.replace('\n', '-------------------------')

In [8]:
df['delete2'] = [(x[-80:-29]) for x in df['delete']]

In [9]:
df['delete3'] = df['delete2'].str.replace('-', '')

In [10]:
df['city'] = df['delete3']

In [11]:
del df['delete']
del df['delete2']
del df['delete3']
del df['location']

In [12]:
###Removing \n from company
df['company'] = df['company'].str.replace('\n', '')

In [13]:
###Removing \n from title
df['title'] = df['title'].str.replace('\n', '')

In [14]:
###Removing \n from description
df['description'] = df['description'].str.replace('\n', '')

In [15]:
###Creating a column for low end of salary
df['low'] = [(x[18:22]) for x in df['salary']]

In [16]:
###Removing '$'
df['low'] = df['low'].str.replace('$', '')

In [17]:
###Removing 'k'
df['low'] = df['low'].str.replace('k', '')

In [18]:
###Removing '-'
df['low'] = df['low'].str.replace('-', '')

In [19]:
###Removing ','
df['low'] = df['low'].str.replace(',', '')

In [20]:
###Creating a column for high end of salary
df['high'] = [(x[-11:-6]) for x in df['salary']]

In [21]:
###Removing '$'
df['high'] = df['high'].str.replace('$', '')

In [22]:
###Removing 'k'
df['high'] = df['high'].str.replace('k', '')

In [23]:
###Removing '-'
df['high'] = df['high'].str.replace('-', '')

In [24]:
###Removing ','
df['high'] = df['high'].str.replace(',', '')

In [25]:
###Fixing '\nFull'
df['high'] = df['high'].str.replace('\nFull', '')

In [26]:
df.dtypes

salary         object
company        object
title          object
description    object
state          object
city           object
low            object
high           object
dtype: object

In [27]:
###START OF CODE TO FIX NANS AND CONVERT TO INTEGERS
dftest = df.copy()

In [28]:
###Turning emptys into nans and dropping
dftest['high'].replace('', np.nan, inplace=True)
dftest['low'].replace('', np.nan, inplace=True)
dftest.dropna(subset=['high'], inplace=True)
dftest.dropna(subset=['low'], inplace=True)

In [29]:
###Turning remaining emptys into 0
dftest['high']=dftest['high'].replace('', '0')
dftest['low']=dftest['low'].replace('', '0', inplace=True)

In [30]:
###Overriding error to return 0s and ints
testing = df['high']

In [31]:
def replace_nulls(x):
    try:
        return int(x)
    except:
        '0'

In [32]:
testing = testing.apply(replace_nulls)

In [208]:
#testing.dropna(inplace=True)

In [33]:
lowtest = df['low']

In [34]:
lowtest = lowtest.apply(replace_nulls)

In [35]:
# lowtest.dropna(inplace=True)

In [36]:
# lowtest

In [37]:
###putting together for a dataframe
salarycolumns = zip(lowtest, testing)

In [38]:
salarycolumnsframe = pd.DataFrame(salarycolumns)

In [39]:
df2 = pd.concat([df, salarycolumnsframe], axis=1)

In [40]:
#Renaming Columns
df2.columns = ('salary', 'company', 'title', 'description', 'state', 'city', 'low', 'high', 'lowsalary', 'highsalary')

In [41]:
###Dropping Columns
del df2['salary']
del df2['low']
del df2['high']

In [42]:
#Getting Rid of Error Salaries
def drop_outliers(x):
    if x > 250:
        return np.nan
    else:
        return x

In [43]:
fixer10 = df2['highsalary']

In [44]:
fixer10 = fixer10.apply(drop_outliers)

In [45]:
#Getting Rid of Error Salaries in the 'Low' Column
def low_outliers(x):
    if x > 5:
        return x
    else:
        return np.nan

In [46]:
lowerfixer = df2['lowsalary']

In [47]:
lowhighfix = zip(lowerfixer, fixer10)

In [48]:
lowhigh = pd.DataFrame(lowhighfix)

In [49]:
df2 = pd.concat([df2, lowhigh], axis=1)

In [50]:
df2['low_salary'] = df2[0]
df2['high_salary'] = df2[1]
#df2.head()

In [51]:
##Dropping Columns
del df2[0]
del df2[1]
del df2['lowsalary']
del df2['highsalary']

In [52]:
#Average Salary
((df2['low_salary'].mean()) + (df2['high_salary'].mean())) / 2

95.08940162882547

In [53]:
###Replacing Nans with Average
def averages(x):
    try:
        return int(x)
    except:
        return 95.0

In [54]:
tester94 = df2['low_salary']

In [55]:
tester94 = tester94.apply(averages)

In [56]:
low_with_average = tester94

In [57]:
###Return the mean for 0.0 cells
def zerofixer(x):
    try:
        return (x / 0) + x
    except:
        return 95.0

In [58]:
low_with_average = low_with_average.apply(zerofixer)

In [59]:
low_testing = tester94

In [60]:
toaddagain = pd.DataFrame(low_testing)

In [61]:
df3 = pd.concat([df2, toaddagain], axis=1)

In [62]:
df4 = df3.copy()

In [63]:
df5 = df2.copy()

In [64]:
###Applying the original 'averages' function directly to the dataframe
df5['low_salary'] = df5['low_salary'].apply(averages)

In [65]:
df5['high_salary'] = df5['high_salary'].apply(averages)

In [66]:
df6 = df5.copy()

In [67]:
###Easier method. Returning the mean if salary is less than $5,000.
def zeros_to_average(x):
    if x < 5:
        return 95.0
    else:
        return x

In [68]:
df5['low_salary'] = df5['low_salary'].apply(zeros_to_average)
df5['average_salary'] = (df5['low_salary'] + df5['high_salary']) / 2

In [69]:
dfreg = df5.copy()

In [70]:
del dfreg['low_salary']
del dfreg['high_salary']

In [71]:
dfreg

,company,title,description,state,city,average_salary
0,Sancus Associates,Associate Data Science,"Associate Data Scientist AdTech $125,000-150,0...",DC,District of Columbia,137.5
1,Analytic Recruiting,"Associate, data Science and market Intelligence","Associate, data Science and market Intelligenc...",NY,New York,100.0
2,Tech Mahindra,BI Data Engineer,Position Title: BI Data Engineer Location: New...,OH,New Albany,95.0
3,Challenge Charter School,6th grade Science & Language Arts Teacher - Mi...,Arizona’s 1st Official Core Knowledge® School;...,AZ,Glendale,41.0
4,CyberCoders,Senior Data Scientist - be the first Data Scie...,This position is open as of 10/18/2016. Senior...,CA,Marina del Rey,175.0
5,Medix,Research Scientist,Research Scientist This Research Scientist wil...,TX,Dallas,60.0
6,BIG WEDNESDAY DIGITAL,Junior Data Scientist - global eCommerce - Ban...,Junior Data Scientist - global eCommerce leade...,MD,Baltimore,97.5
7,Burnett Specialists,Data Scientist,Job Reference #: 41-1014TBD-16CT Data Scientis...,TX,Houston,100.0
8,BIG WEDNESDAY DIGITAL,Senior/Lead/NLP- Data Scientist - Bangkok role...,Senior/Lead/NLP- Data Scientist - Bangkok base...,WA,Seattle,97.5
9,Kelly Scientific Resources,"Sr. Manager, Medical Data Methodology",Kelly Clinical and a highly reputable pharmace...,MA,Cambridge,110.0


In [72]:
##Bucketing Income
def bucket(x):
    if x['average_salary'] > 95:
        return 1
    else:
        return 0

In [73]:
dfreg2 = dfreg.copy()

In [74]:
dfreg2['high_vs_low'] = dfreg2.apply(bucket, axis =1)

In [75]:
##Now have complete dataset
dfreg2['label'] = dfreg2['high_vs_low']

In [76]:
del dfreg2['high_vs_low']

In [77]:
data = dfreg2

<h1> 'Data' is our final dataset </h1>
<br>
<p>
After scraping 30 pages of search results for "Data Science" on careerbuilder.com, cleaning the data, and creating a binary column for 'high' vs 'low' salary we finally have our final dataframe. We determined salaries above the mean of $95,000 would be considered 'high' and anything else would be 'low'.
</p>

In [78]:
data.head()

,company,title,description,state,city,average_salary,label
0,Sancus Associates,Associate Data Science,"Associate Data Scientist AdTech $125,000-150,0...",DC,District of Columbia,137.5,1
1,Analytic Recruiting,"Associate, data Science and market Intelligence","Associate, data Science and market Intelligenc...",NY,New York,100.0,1
2,Tech Mahindra,BI Data Engineer,Position Title: BI Data Engineer Location: New...,OH,New Albany,95.0,0
3,Challenge Charter School,6th grade Science & Language Arts Teacher - Mi...,Arizona’s 1st Official Core Knowledge® School;...,AZ,Glendale,41.0,0
4,CyberCoders,Senior Data Scientist - be the first Data Scie...,This position is open as of 10/18/2016. Senior...,CA,Marina del Rey,175.0,1


In [91]:
def examine_coefficients(model, dftitles):
    dftitles = pd.DataFrame(
        { 'Coefficient' : model.coef_[0] , 'Feature' : dftitles.columns}
    ).sort_values(by='Coefficient')
    return dftitles[dftitles.Coefficient !=0 ]

In [97]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(
    binary=True,  
    stop_words='english', 
    max_features=50, 
)



X = v.fit_transform(data.title).todense()
X = pd.DataFrame(X, columns=v.get_feature_names())
X.head()

,administrator,analyst,analytics,application,applications,architect,associate,bangkok,bi,business,...,software,specialist,sql,sr,supervisor,support,systems,technical,technology,web
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
y = data.label

In [100]:
model = LogisticRegression(penalty = 'l1', C = 10.0) 

model.fit(X, y)
datatitle = examine_coefficients(model, X)

In [129]:
datatitle

,Coefficient,Feature
27,-4.429640,level
10,-3.023888,coordinator
2,-1.147153,analytics
18,-0.927597,electrical
6,-0.848606,associate
35,-0.841610,quality
34,-0.506156,project
1,-0.487379,analyst
45,-0.448440,support
17,-0.443106,ecommerce


In [103]:
###Testing to see how well the model performed
from sklearn.cross_validation import cross_val_score

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
    scores = cross_val_score(model, X, y, scoring=metric)
    print("mean {}: {}, all: {}".format(metric, scores.mean(), scores))

mean accuracy: 0.664, all: [ 0.696  0.624  0.672]
mean precision: 0.664378411951, all: [ 0.70707071  0.61616162  0.66990291]
mean recall: 0.569800569801, all: [ 0.5982906   0.52136752  0.58974359]
mean roc_auc: 0.733896707581, all: [ 0.75049804  0.6916008   0.75959129]


<h1> Testing the Model </h1>
<br>
<p>
Our model testing words in 'job title' to predict salary had an roc_auc score of 0.75, accuracy of .66, precision of .66, and recall of .57.
</p>
<p> The words in the job title that were the strongest indicators of low salary were level, coordinator, and analytics.
</p>
<p>
The words in the job title that were the strongest indicators of high salaries were infrastructure, architect, director, and engineering.
</p>
<p>
The words in job description that were the strongest indicators of high salaries were 'manager', 'responsible', 'experience', 'senior', 'business',  and 'growing'.
</p>
<p>
The words in job description that were the strongest indicators of low salaries were 'analyst', 'position', 'job', 'support', 'opportunity', 'hire', 'new', 'systems', and 'time'.
</p>

In [108]:
def examine_coefficients2(model, dfdesc):
    dfdesc = pd.DataFrame(
        { 'Coefficient' : model.coef_[0] , 'Feature' : dfdesc.columns}
    ).sort_values(by='Coefficient')
    return dfdesc[dfdesc.Coefficient !=0 ]

In [109]:
###Model based on Description
v2 = CountVectorizer(
    binary=True,  # Create binary features
    stop_words='english', # Ignore common words such as 'the', 'and'
    max_features=50, # Only use the top 50 most common words
)


# This builds a matrix with a row per website (or data point) and column per word (using all words in the dataset)
X2 = v2.fit_transform(data.description).todense()
X2 = pd.DataFrame(X2, columns=v2.get_feature_names())
X2.head()

,00,000,10,18,2016,analyst,based,business,classification,client,...,solutions,summary,support,systems,team,technical,technology,time,work,year
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [110]:
y = data.label

In [111]:
model = LogisticRegression(penalty = 'l1', C = 1.0) # Model with high, little regularization

model.fit(X2, y)
datadescription = examine_coefficients(model, X2)

In [113]:
#datadescription

In [114]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix

import numpy as np
import pandas as pd
import patsy

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.grid_search import GridSearchCV

In [115]:
###We also created a model based on City/State/Title/Description
X = patsy.dmatrix('~C(city) + C(state) +C(company) + C(title) +C(description)' , data)
y = data['label'].values

In [116]:
pdf = pd.DataFrame(X, columns=X.design_info.column_names)
pdf['Target'] = y
#pdf

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=77)

In [118]:
lr = LogisticRegression(solver='liblinear')

In [119]:
lr_model = lr.fit(X_train, y_train)

In [120]:
lr_ypred = lr_model.predict(X_test)

In [121]:

lr_cm = confusion_matrix(y_test, lr_ypred, labels=lr.classes_)
lr_cm = pd.DataFrame(lr_cm, columns=lr.classes_, index=lr.classes_)
lr_cm

,0,1
0,98,33
1,55,62


In [123]:
print classification_report(y_test, lr_ypred, labels=lr.classes_)

             precision    recall  f1-score   support

          0       0.64      0.75      0.69       131
          1       0.65      0.53      0.58       117

avg / total       0.65      0.65      0.64       248



In [124]:
lr_l1 = LogisticRegression(C=500, penalty='l1', solver='liblinear')
lr_l1_model = lr_l1.fit(X_train, y_train)

In [125]:
lr_l1_model = lr_l1.fit(X_train, y_train)

In [126]:
lr_l1_ypred = lr_l1_model.predict(X_test)

In [127]:
lr_l1_cm = confusion_matrix(y_test, lr_l1_ypred, labels=lr_l1.classes_)
lr_l1_cm = pd.DataFrame(lr_l1_cm, columns=lr_l1.classes_, index=lr_l1.classes_)
lr_l1_cm

,0,1
0,113,18
1,62,55


In [128]:
print classification_report(y_test, lr_l1_ypred, labels=lr_l1.classes_)

             precision    recall  f1-score   support

          0       0.65      0.86      0.74       131
          1       0.75      0.47      0.58       117

avg / total       0.70      0.68      0.66       248



<p>
Our model testing based on city, state, title, description, and company had an f1-score of .64 when we did a regular logistic regression and had an f1-score of .66 when we used an L1 with a C of 500.
</p>